# Interatomic force field (IFF) optimization for W-based metal alloys

## Calculate configuration statistics for EAM models

Author: Lukas Vlcek

Start Date: 2018-04-22

In [1]:
from datetime import datetime ; print('Last update:', datetime.now())

Last update: 2018-08-09 16:43:21.215586


In [2]:
%matplotlib inline
import os
import re
import numpy as np
import pickle
import matplotlib.pyplot as plt

# source of raw data and destination for processed data
target_proc = '../data/target_processed'

**Read a list of trajectory datasets for processing**

In [3]:
with open(os.path.join(target_proc,  "datasets.pickle"), 'rb') as fi:
    datasets = pickle.load(fi)
    print(datasets)

['bcc_npt_langevin_300K', 'i110_npt_langevin_2000K', 'bcc_npt_langevin_3700K', 'liq_5000K', 'screw_111_npt_langevin_2000K', 'fcc_npt_langevin_300K']


In [5]:
trajectories = []
for dset in datasets:
    with open(os.path.join(target_proc, dset + ".pickle"), 'rb') as fo:
        trajectories.append(pickle.load(fo))

Function to calculate atom pair distances in a triclinic box

In [13]:
def pair_dist_triclinic(xyz, box):
    """
    Calculates nearest image pair distances between all atoms in xyz array.
    Parameters
    -----------
    xyz : numpy array
          particle x, y, z coordinates
    box : numpy 2D array of unit cell vectors
          simulation box dimensions/shape
    Returns
    -------
    rr  : (natom, natom) numpy array of pair distances
    rx  : (natom, natom, 3) numpy array of pair distance coordinates
    """

    n_atom = xyz.shape[0] # number of atoms in a configuration
    rr = np.empty((n_atom, n_atom), dtype=float)
    rx = np.empty((n_atom, n_atom, 3), dtype=float)
    
    boxT = box.T
    
    for i, pa in enumerate(xyz):
        for j, pb in enumerate(xyz):
            dp = pa - pb
            dp = np.where(dp < -0.5, dp + 1.0, dp)
            dp = np.where(dp >  0.5, dp - 1.0, dp)
            
            dp = boxT.dot(dp)
            
            rr[i,j] = np.sum(dp*dp)**0.5
            rx[i,j] = dp

    return rr, rx

In [11]:
# sufficient statistics for EAM
def get_stats_EAM(rr, rx, sc):
    """
    Takes atom pair distances and calculates sufficeint statistics needed
    for the parameterization of a cubic spline-based EAM model by Bonny et al. (2017).
    
    Parameters
    ----------
    rr : numpy array
         set of pair distances
    rx : numpy array
         set of pair distance coordinates
    sc : python list
         spline nodes
         
    Returns
    -------
    ar, a1, a2 : numpy arrays (len(sc))
                 atom energy-related statistics
                 el_density**0.5, el_density, el_density**2
    br, b1, b2 : numpy arrays (len(sc), natoms, 3 coordinates)
                 atom force-related statistics (gradients of energy)
                 grad(el_density**0.5), grad(el_density), grad(el_density**2)
    """

    n_atom = rr.shape[0]

    # energy-related statistics
    aa = np.empty((n_atom), dtype=float)
    ar = np.zeros((len(sc)), dtype=float)
    a1 = np.zeros_like(ar)
    a2 = np.zeros_like(ar)

    # force-related statistics
    br = np.zeros((len(sc), n_atom, 3), dtype=float)
    b1 = np.zeros_like(br)
    b2 = np.zeros_like(br)
    zero3 = np.zeros((3))

    # cycle over spline nodes
    for ks, rc in enumerate(sc):

        # cycle over atoms
        for i in range(n_atom):

            # sum electronic density over all neighbors of i within rc
            aa[i] = sum([(rc - r)**3 for r in rr[i] if (r < rc and r > 0.01)])

            # if el. density larger than zero, calculate force statistics                                                                                                                                        84,0-1        43%
            if aa[i] > 0.0:

                # precompute a list of recurring values for force statistics
                ff = [1.5*(rc - r)**2*x/r if (r > 0.01 and r < rc) else zero3 for r, x in zip(rr[i], rx[i])]

                # sum contributions to force statistics from all neighbors of i
                b1[ks, i] = sum([2*f       for f in ff])
                br[ks, i] = sum([ -f/np.sqrt(aa[i]) for f in ff])
                b2[ks, i] = sum([4*f*aa[i] for f in ff])

        # sum contributions to energy statistics for a given spline node
        ar[ks] = np.sum(np.sqrt(aa))
        a1[ks] = np.sum(aa)
        a2[ks] = np.sum(aa**2)

    return a1, ar, a2, b1, br, b2

**Calculate pair distances and energy statistics**

Force statistics not calculated for longer MD trajectories.

In [22]:
sc = [2.56, 2.73, 3.252, 3.804, 4.20, 4.77]
weights = [1.0 for _ in range(len(trajectories))]

stats_data = {}
target_data = {}

for di, (traj, weight) in enumerate(zip(trajectories, weights)):
    
    # target data
    target_dict = {'type':'trajectory', 'weight':weight}
    target_dict['box'] = traj['box']
    target_dict['xyz'] = traj['xyz']
    target_dict['energy'] = traj['energy']
    target_dict['temp'] = traj['temp']

    # save inverse temperature data (if T=0, set beta=1/300)
    target_dict['beta'] = np.empty_like(target_dict['temp'])
    for i, temp in enumerate(target_dict['temp']):
        if temp == 0.0:
            target_dict['beta'][i] = 1.0/300.0
        else:
            target_dict['beta'][i] = 1.0/temp
            
    target_data['dset'+str(di)] = target_dict

    # statistics data
    stats_dict = {'energy':[]}
    for xyz, box in zip(traj['xyz'], traj['box']):
        rr, rx = pair_dist(xyz, box)
        
        a1, ar, a2, f1, fr, f2 = get_stats_EAM(rr, rx, sc)
        #print(xyz.shape, box)
        #print('x', a1.shape, rr.shape, np.sum(np.abs(a1)))

        stats_dict['energy'].append(np.array([ar, a2, a1]))
        #stats_dict['forces'].append(np.array([fr, f2, f1]))
               # add dataset
            
    stats_data['dset'+str(di)] = stats_dict
    print('dataset #', di)

# pickle stats and target data to be used for optimization
with open('../../../data/working/target_2.pickle', 'wb') as fo:
    pickle.dump(target_data, fo)

with open('../../../data/working/stats_2.pickle', 'wb') as fo:
    pickle.dump(stats_data, fo)

KeyboardInterrupt: 

Make combined reference datasets